In [1]:
from autocnet_server.utils.add_image import ImageAdder

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Adding images to the database

The ImageAdder object is a job dispatcher that has two primary access methods: `add_image` and `add_images`.  The former adds a single image to the database and the latter accepts both a directory and a file extension.  All files with the extension in said directory will be added to the database.  The process of adding an image includes the extraction of interest points (features), serialization of the interest points to disk in the same location as the image (saved as a .h5 file), generation of a camera model, computation of the image footprint, and addition of this information to the database.

When the object is created a message will print indicating that a PYRO server is also running:

```
Object <autocnet_server.utils.add_image.ImageAdder object at 0x2ae8b549b9e8>:
    uri = PYRO:ia@0.0.0.0:8005
Pyro daemon running.
```

This is entirely normal.  NAC works asynchronously, in that jobs are spawned to the cluster without locking up the jupyter notebook that can be used to work with NAC objects. The NAC objects create a listening server on a different thread and register callback functions so that, once a cluster job has completed, the NAC objects can be alerted.  This allows the NAC objects to manage the database connection and not swamp the database with hundreds of concurrent connections.

For now, the port (8005) is hard coded.  If the next cell is run twice, without restarting the jupyter kernel, an error will be thrown that port (8005) is already in use.  In the menus at the top of this notebook, select 'Kernel->Restart' to restart the kernel and free the port.

In [2]:
ia = ImageAdder()

Object <autocnet_server.utils.add_image.ImageAdder object at 0x2af674eeb128>:
    uri = PYRO:ia@0.0.0.0:8005
Pyro daemon running.


## Add Images
Here a directory of images is added to the database. The `add_images` methods takes two arguments: (1) the directory NS (2) the file extension.  All files in the directory that have the file extension will be added to the database table (defined) above in the `d={}` cell.

In [4]:
res = ia.add_images('/scratch/jlaura/ctx/cal/', 'cub')

## Add a single Image
and force the keypoints to be reextracted

In [3]:
ia.add_image('/scratch/jlaura/ctx/cal/J11_048958_1874_XN_07N207W.cal.cub', force=True)
ia.add_image('/scratch/jlaura/ctx/cal/J02_045543_1866_XN_06N207W.cal.cub', force=True)

In [5]:
ia.config.database_name

'test4'

## Job Status

Knowing that a job has been submitted to a cluster is not all that helpful.  What matters is that the job completed successfully.  The `ia` object has two attributes to help with knowing what is going on with the jobs.  The first is `ia.job_status` that returns a dictionary with the key being the image name and the value being the either `True` (everything is working great and the image has be processed successfully) or a number (1,2,3,4).  If the value is a number, it means that the job has failed $n$ times so far.  We let a job fail 3 times before giving up on that image.

In [5]:
ia.job_status

defaultdict(dict,
            {'/scratch/jlaura/ctx/cal/B02_010414_1897_XN_09N202W.cal.cub': {'count': 0,
              'submission': '#!/bin/bash -l\n#SBATCH -n 1\n#SBATCH --mem-per-cpu 8192\n#SBATCH -J AutoCNet\n#SBATCH -t 01:00:00\n#SBATCH -o /home/jlaura/logs/860872.log\n#SBATCH -p shortall\n#SBATCH --exclude=neb[13-20],gpu1\n/home/jlaura/anaconda3/envs/ct/bin/python /home/jlaura/autocnet_server/bin/extract_features.py /scratch/jlaura/ctx/cal/B02_010414_1897_XN_09N202W.cal.cub PYRO:ia@neb4:8005',
              'success': True}})

The above is along list with (hopefully) lots of True values.  What is really of interest are the issues where keypoints could not be extracted.  To see those, the image_adder has a failed jobs attribute.

The output below is a little bit lame - an empty list indicating that everything worked.  Let's pretend for a minute that the list contained `'/scratch/jlaura/ctx/cal/B01_009900_1837_XN_03N206W.cal.cub'`.  We might want to inspect this job to see what happened.

In [15]:
ia.failed_jobs

['/scratch/jlaura/ctx/cal/B02_010414_1897_XN_09N202W.cal.cub',
 '/scratch/jlaura/ctx/cal/B20_017350_1888_XN_08N202W.cal.cub',
 '/scratch/jlaura/ctx/cal/G19_025644_1890_XN_09N199W.cal.cub',
 '/scratch/jlaura/ctx/cal/B17_016229_1890_XN_09N197W.cal.cub',
 '/scratch/jlaura/ctx/cal/B16_016018_1875_XI_07N195W.cal.cub',
 '/scratch/jlaura/ctx/cal/P17_007777_1943_XN_14N201W.cal.cub',
 '/scratch/jlaura/ctx/cal/F17_042391_1881_XN_08N198W.cal.cub',
 '/scratch/jlaura/ctx/cal/D20_035033_1882_XN_08N210W.cal.cub',
 '/scratch/jlaura/ctx/cal/D04_028954_1892_XN_09N201W.cal.cub',
 '/scratch/jlaura/ctx/cal/P06_003373_1885_XN_08N199W.cal.cub',
 '/scratch/jlaura/ctx/cal/J03_045965_1894_XN_09N209W.cal.cub',
 '/scratch/jlaura/ctx/cal/P14_006472_1830_XN_03N206W.cal.cub',
 '/scratch/jlaura/ctx/cal/D07_029864_1878_XN_07N205W.cal.cub',
 '/scratch/jlaura/ctx/cal/G21_026290_1877_XI_07N197W.cal.cub',
 '/scratch/jlaura/ctx/cal/P16_007144_1926_XN_12N196W.cal.cub',
 '/scratch/jlaura/ctx/cal/P19_008265_1859_XN_05N205W.ca

In [6]:
ia.job_status['/scratch/jlaura/ctx/cal/B01_009900_1837_XN_03N206W.cal.cub']

True

## Rerunning Failed Jobs
